In [1]:
import pandas as pd

train_df = pd.read_csv('A3_task1_data_files/train.csv', sep='\t')
dev_df = pd.read_csv('A3_task1_data_files/dev.csv', sep='\t')

dev_df.rename(columns={'setence1': 'sentence1'}, inplace=True)

In [2]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from transformers import BertTokenizer
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')


/home/arnav/miniconda3/envs/DLA2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/arnav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/arnav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/arnav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/arnav/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Punctuation removal
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])

    return lemmatized_text

In [4]:
class Task1A_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text1 = str(self.dataframe.iloc[idx]['sentence1'])
        text2 = str(self.dataframe.iloc[idx]['sentence2'])

        sentence1 = preprocess_text(text1)
        sentence2 = preprocess_text(text2)

        score = self.dataframe.iloc[idx]['score']

        # Tokenize the pair of sentences to get the token ids, attention masks, and token type ids
        encoding = self.tokenizer.encode_plus(
            sentence1, sentence2,
            add_special_tokens=True, #cls and sep
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(score, dtype=torch.float)
        }

In [5]:
# Assuming 'df' is your DataFrame
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize the dataset
train_dataset = Task1A_Dataset(train_df, tokenizer)
dev_dataset = Task1A_Dataset(dev_df, tokenizer)

# Create a DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=64, shuffle=False)

In [6]:
from transformers import BertModel
import torch
import torch.nn as nn

class BertForTextSimilarity(nn.Module):
    def __init__(self, freeze_bert=False):
        super(BertForTextSimilarity, self).__init__()
        # Load pre-trained BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Add a linear layer for regression
        self.regression = nn.Linear(self.bert.config.hidden_size, 1)

        # Option to freeze BERT layers to prevent them from being updated during training
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        # Get the output from BERT model
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # The first token of every sequence is a special token ([CLS]) that contains the aggregate representation for classification tasks. We use it for regression here.
        cls_output = outputs.last_hidden_state[:, 0, :]  # Shape: (batch_size, hidden_size)

        # Pass the [CLS] token's output through the regression layer
        score = self.regression(cls_output)  # Shape: (batch_size, 1)

        return score


In [7]:
model = BertForTextSimilarity()
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)  # Move model to CUDA device if available
    print("Using CUDA:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU instead.")

Using CUDA: NVIDIA RTX A5000


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
mse_loss = torch.nn.MSELoss()

num_epochs = 3  # or however many epochs you plan to train for

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    train_loss = 0
    i = 0
    for batch in train_dataloader:
        print(i)
        i+=1
        # Forward pass
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        loss = mse_loss(outputs.squeeze(), inputs['labels'])

        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    # After each epoch, do validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    with torch.no_grad():  # No need to compute gradients during validation
        for batch in dev_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
            loss = mse_loss(outputs.squeeze(), inputs['labels'])
            val_loss += loss.item()

    # Calculate average losses
    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(dev_dataloader)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

0
1


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


...

62
63
64
65
66
67
68
69
70
71


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch 3/3, Train Loss: 0.4429, Validation Loss: 0.7224


In [9]:
# Calculate similarity scores on the validation set

model.eval()  # Set the model to evaluation mode
predictions = []
labels = []

with torch.no_grad():  # No need to compute gradients during validation
    for batch in dev_dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        predictions.extend(outputs.squeeze().tolist())
        labels.extend(inputs['labels'].tolist())

# Calculate Pearson correlation
from scipy.stats import pearsonr

correlation, p_value = pearsonr(labels, predictions)

print(f'Pearson Correlation: {correlation:.4f}, P-Value: {p_value:.4f}')

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Pearson Correlation: 0.8401, P-Value: 0.0000


In [10]:
print(predictions)

[4.693336009979248, 4.7085442543029785, 4.358657360076904, 3.1744496822357178, 3.671196460723877, 3.614952802658081, 4.799011707305908, 2.4625680446624756, 4.583574295043945, 4.747319221496582, 3.5506203174591064, 2.159536838531494, 4.74253511428833, 4.858457088470459, 4.544525623321533, 0.8718488812446594, 3.7103030681610107, 4.8556132316589355, 4.238290309906006, 1.1416125297546387, 3.2388880252838135, 1.878218412399292, 3.5133578777313232, 1.922269344329834, 0.9625773429870605, 3.0363857746124268, 3.8730363845825195, 3.7058303356170654, 0.883797287940979, 0.2812442481517792, 4.1213483810424805, 4.435329914093018, 4.539696216583252, 4.50732421875, 4.3185625076293945, 4.561171054840088, 4.5720295906066895, 4.581595420837402, 4.069953441619873, 1.1741275787353516, 3.9322822093963623, 4.195449352264404, 2.73103666305542, 4.084671974182129, 1.4325447082519531, 0.43928900361061096, 0.04780564829707146, 0.18695789575576782, 2.929619073867798, 4.169259071350098, 0.5952378511428833, 4.785333

In [11]:
print(labels)

[5.0, 4.75, 5.0, 2.4000000953674316, 2.75, 2.615000009536743, 5.0, 2.3329999446868896, 3.75, 5.0, 3.200000047683716, 1.5829999446868896, 5.0, 5.0, 4.908999919891357, 0.800000011920929, 2.4000000953674316, 5.0, 4.0, 0.6359999775886536, 3.0, 1.7139999866485596, 3.200000047683716, 2.1670000553131104, 1.0, 1.9170000553131104, 4.25, 3.0, 1.0, 0.6000000238418579, 2.5999999046325684, 5.0, 4.599999904632568, 5.0, 4.800000190734863, 3.799999952316284, 5.0, 5.0, 4.199999809265137, 1.399999976158142, 3.5999999046325684, 2.799999952316284, 1.600000023841858, 3.0, 1.399999976158142, 0.25, 0.25, 0.0, 4.0, 4.5, 0.5, 3.799999952316284, 4.800000190734863, 5.0, 0.25, 1.2000000476837158, 0.6000000238418579, 0.800000011920929, 3.799999952316284, 0.0, 3.5, 4.5, 2.799999952316284, 3.799999952316284, 3.799999952316284, 0.0, 4.0, 4.25, 2.812000036239624, 4.25, 3.0, 1.0, 3.75, 0.0, 0.4000000059604645, 4.0, 2.799999952316284, 3.75, 1.1540000438690186, 2.75, 2.799999952316284, 0.0, 0.0, 3.4000000953674316, 5.0, 